In [3]:
import numpy as np
import pandas as pd
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup

# scrap
url = 'https://pokemondb.net/pokedex/all'
agent = Request(url, headers={'User-Agent':'Mozilla/5.0'})
html = urlopen(agent).read()
data = BeautifulSoup(html, 'html.parser')

In [62]:
# ambil data
table = data.find_all('table', class_='data-table')
rows = table[0].find_all('tr')

# ambil data tiap sel
hasil = []
for row in rows:
    isi = []
    for cell in row.find_all(['th', 'td']):
        isi_ = cell.get_text(strip=True).replace(' ','')
        isi.append(isi_)
    if 'Diglett' not in isi:
        hasil.append(isi)
    else:
        break
hasil.pop(0)

['#',
 'Name',
 'Type',
 'Total',
 'HP',
 'Attack',
 'Defense',
 'Sp.Atk',
 'Sp.Def',
 'Speed']

In [63]:
hasil_fin = hasil

In [64]:
df = pd.DataFrame(hasil_fin)
df.head()

,0,1,2,3,4,5,6,7,8,9
0,001,Bulbasaur,GrassPoison,318,45,49,49,65,65,45
1,002,Ivysaur,GrassPoison,405,60,62,63,80,80,60
2,003,Venusaur,GrassPoison,525,80,82,83,100,100,80
3,003,VenusaurMegaVenusaur,GrassPoison,625,80,100,123,122,120,80
4,004,Charmander,Fire,309,39,52,43,60,50,65


In [75]:
# buat database
import sqlite3
conn = sqlite3.connect('pokemon.db')
cur = conn.cursor()

In [76]:
# add table
create_table_data = '''CREATE TABLE IF NOT EXISTS pokemon(
                  Class INT(3),
                  Name TEXT PRIMARY KEY,
                  Type TEXT,
                  Total INT,
                  HP INT,
                  Attack INT,
                  Defense INT,
                  SpAtk INT,
                  SpDef INT,
                  Speed INT);'''
cur.execute(create_table_data)

In [77]:
# add values to table
queries = '''INSERT INTO pokemon 
            VALUES(?,?,?,?,?,?,?,?,?,?)'''
cur.executemany(queries, hasil_fin)
conn.commit()

In [79]:
cur.execute('SELECT * FROM pokemon;')
three_results = cur.fetchmany(3)
print(three_results)

[(1, 'Bulbasaur', 'GrassPoison', 318, 45, 49, 49, 65, 65, 45), (2, 'Ivysaur', 'GrassPoison', 405, 60, 62, 63, 80, 80, 60), (3, 'Venusaur', 'GrassPoison', 525, 80, 82, 83, 100, 100, 80)]


In [80]:
cur.close()
conn.close()